In [1]:
EXAMPLE_INPUT = 'RENE=MO10:00-12:00,TU10:00-12:00,TH01:00-03:00,SA14:00-18:00,SU20:00-21:00'

In [2]:
EXAMPLE_INPUT = 'ASTRID=MO10:00-12:00,TH12:00-14:00,SU20:00-21:00'

In [3]:
import re
from datetime import time, datetime, date

In [4]:
re.findall("(\w{2})(\d{2}:\d{2})-(\d{2}:\d{2})", EXAMPLE_INPUT)

[('MO', '10:00', '12:00'), ('TH', '12:00', '14:00'), ('SU', '20:00', '21:00')]

In [5]:
from datetime import time
from typing import Tuple
Hour = Tuple[int, int]
class Interval:
    def __init__(self, inital_time : Hour, final_time : Hour) -> None:
        if(final_time[0] == 0 and final_time[1] ==0):
            final_time[0] = 24
        assert(final_time > inital_time)
        self.initial_time = inital_time
        self.final_time = final_time
        
    def eliminate_minutes_from_hour(self, hour : Hour) -> float:
        return hour[0] + hour[1]/60

    def in_interval(self, start_time: Hour, end_time : Hour):
        
        
        return (
            (self.eliminate_minutes_from_hour(start_time) - self.eliminate_minutes_from_hour(self.initial_time)) >=0) and (
            self.eliminate_minutes_from_hour(end_time) - self.eliminate_minutes_from_hour(self.final_time) <=0
        )






In [6]:
## Default intervals
first_interval = Interval((0, 1),  (9, 0))
second_interval = Interval((9, 1), (18, 0))
third_interval = Interval((18, 1), (24, 0))

intervals = [first_interval, second_interval, third_interval]

In [7]:
from enum import Enum

class DefaultTagEnum(Enum):
    WeekDay = 0
    Weekend = 1

In [8]:
payment_configuration= {
    DefaultTagEnum.WeekDay : {
        first_interval : 25,
        second_interval : 15,
        third_interval : 20
    }, 
    
    DefaultTagEnum.Weekend: {
        first_interval : 30,
        second_interval : 20,
        third_interval : 25
    }
}

In [9]:
RE_PATTERN ="(\w{2})(\d{2}:\d{2})-(\d{2}:\d{2})"
def parse_times(current_input : str) -> Tuple[str, Hour, Hour]:
    for data in re.findall(RE_PATTERN, EXAMPLE_INPUT):
        day, start, end = data
        
        start_hour, start_minute = start.split(':')
        start_hour, start_minute = int(start_hour), int(start_minute)
    
        end_hour, end_minute = end.split(':')
        end_hour, end_minute = int(end_hour), int(end_minute)
        
        yield (day, (start_hour, start_minute), (end_hour, end_minute))
        
    

In [10]:
def get_total_earned_on_day(input_str: str):
    total_earned = 0
    for day, start_time, end_time in parse_times(input_str):
        period_worked_hours = end_time[0] - start_time[0] + (end_time[1] - start_time[1])/60

        matched_intervals = [interval for interval in intervals if interval.in_interval(start_time, end_time)]

        assert(len(matched_intervals) == 1)

        matched_interval = matched_intervals[0]

        if(re.match('MO|TU|WE|TH|FR', day)):
            total_earned += payment_configuration[DefaultTagEnum.WeekDay][matched_interval] * period_worked_hours

        elif (re.match('SA|SU', day)):
            total_earned += payment_configuration[DefaultTagEnum.Weekend][matched_interval] * period_worked_hours

        else:
            raise ValueError("Day does not exist")
            
    return total_earned
    
get_total_earned_on_day(EXAMPLE_INPUT)

85.0

total_earned = 0
for data in re.findall("(\w{2})(\d{2}:\d{2})-(\d{2}:\d{2})", EXAMPLE_INPUT):
    day, start, end = data
    start_hour, start_minute = start.split(':')
    start_hour, start_minute = int(start_hour), int(start_minute)
    
    
    end_hour, end_minute = end.split(':')
    end_hour, end_minute = int(end_hour), int(end_minute)
    if(end_hour == 0):
        end_hour = 24
    
    period_worked_hours = end_hour - start_hour
    
    
    matched_interval = [i for i in  intervals if i.in_interval( 
            (start_hour, start_minute), (end_hour, end_minute)
    )]
    
    assert(len(matched_interval) == 1)
    
    matched_interval = matched_interval[0]
    
    
    if(re.match('MO|TU|WE|TH|FR', day)):
        total_earned += payment_configuration[DefaultTagEnum.WeekDay][matched_interval] * period_worked_hours
        
    elif (re.match('SA|SU', day)):
        total_earned += payment_configuration[DefaultTagEnum.Weekend][matched_interval] * period_worked_hours
        
    else:
        print("_______ERRROR______")
    
    print(data)
    print(third_interval.in_interval((start_hour, start_minute), (end_hour, end_minute)))

total_earned

time(0, 0) == time(0,0)

In [11]:
from DataStructures.DefaultTagEnum import DefaultTagEnum
from DataStructures.Interval import Interval

class ConfigManager:
    default_tag_price : dict[DefaultTagEnum, dict[Interval, int]] =None
        # DefaultTagEnum.WeekDay : [25,15,20],
        # DefaultTagEnum.Weekend : [30, 20, 25]
    # }

    intervals = []


    custom_tag_price : dict[str, dict[Interval, int]] = {}

    @staticmethod
    def register_tag_price(tag : str,prices_by_interval: dict[Interval, int]):
        ConfigManager.custom_tag_price[tag] = prices_by_interval

    @staticmethod
    def register_interval(interval : Interval):
        if(ConfigManager.default_tag_price is not None): raise ValueError("You can't register any more intervals as the configuration already began")
        if(interval in ConfigManager.intervals): raise ValueError("Interval is already registered")

        overrided_intervals = [i for i in ConfigManager.intervals if i.in_interval(interval.initial_time, interval.initial_time) or i.in_interval(interval.final_time, interval.final_time)]
        
        if(len(overrided_intervals) > 0):
            print(overrided_intervals)
            raise ValueError("Interval is overriding one already registered")
        ConfigManager.intervals.append(interval)
        

    @staticmethod
    def register_configuration(config : dict[DefaultTagEnum, dict[Interval, int]]):
        for tag in config:
            configured_intervals = [*config[tag].keys()]
            
        assert(set(configured_intervals) == set(ConfigManager.intervals))
                




In [12]:
ConfigManager.intervals = []
print(ConfigManager.intervals)

[]


In [13]:
ConfigManager.register_interval(first_interval)

In [14]:
ConfigManager.register_interval(second_interval)

In [15]:
ConfigManager.register_interval(third_interval)

In [16]:
ConfigManager.intervals

In [17]:
ConfigManager.register_configuration(
    {DefaultTagEnum.WeekDay : {
        first_interval:2,
        second_interval: 3,
        third_interval: 4
    }
     })

# Dependency injection

In [18]:
from inspect import signature
from inspect import _empty

In [70]:
class IMyClass:
    pass

class MyClassDependency (IMyClass):
    def __init__(self ):
        pass

class IClassDependency:
    pass

class ClassToDependencyInject (IMyClass):
    def __init__(self, MyClassDependency : MyClassDependency, my_number : int = 2):
        self.my_class_dependency = MyClassDependency

In [71]:
def get_dependencies_of_classes( *args):
    dependency_mapping = {}
    for class_to_instantiate in args:
        s = signature(class_to_instantiate.__init__)
        dependency_mapping[class_to_instantiate] = []
        for p in s.parameters:    
            if(p == 'self'): continue
            annotation = s.parameters[p].annotation
            if(annotation is not _empty):
                dependency_mapping[class_to_instantiate].append(annotation)
            
    return dependency_mapping



In [72]:
get_dependencies_of_classes(MyClassDependency, ClassToDependencyInject)

{__main__.MyClassDependency: [],
 __main__.ClassToDependencyInject: [__main__.MyClassDependency, int]}

In [97]:
class DependencyInjectionContainer:
    '''
        This is a simple implementation of the dependency injection pattern. It's not very robust as it is not necessary for the project.
        Usually would be better to do it with a whole graph library and search for loops but I'm keeping it simple.
    '''
    DependencyInjectedClass = any
    def __init__(self, *args : DependencyInjectedClass):
        self.dependency_injected_classes : list = [*args] #convert tuple to list
        self.get_dependencies_of_classes()
        self.__check_for_missing_dep()
        self.__check_for_loops()
        self.instances = {}

        self.__instantiate_classes()

    def __check_for_missing_dep(self):
        all_dependencies = []
        for class_to_inject in self.dependency_mapping:
            all_dependencies += self.dependency_mapping[class_to_inject]

        if not set(all_dependencies).issubset(self.dependency_mapping.keys()):


            all_dep_set = set(all_dependencies)
            dependency_map_set = set([*self.dependency_mapping.keys()])

            raise ValueError("Some dependencies are not inside the container")

    def __check_for_loops(self):

        #This function is extremly unoptimized but it'll do for now. 
        #If I have time I'll change it for better one.
        for class_to_inject in self.dependency_mapping:
            for dependency in self.dependency_mapping[class_to_inject]:
                if class_to_inject in self.dependency_mapping[dependency]:
                    raise ValueError("There is a loop in your dependencies")

    def __instantiate_classes(self):
        while len(self.dependency_injected_classes) != 0:    
            for classe_to_inject in self.dependency_injected_classes:
                if(len(self.dependency_mapping[classe_to_inject]) == 0):
                    self.instances[classe_to_inject] = classe_to_inject()
                    print(f"Instantiating {classe_to_inject}")
                    self.dependency_injected_classes.remove(classe_to_inject)

                elif(set(self.dependency_mapping[classe_to_inject]).issubset(self.instances.keys())):
                    dependency_instances_list = [self.instances[c] for c in self.dependency_mapping[classe_to_inject]]
                    print(f"Instantiating {classe_to_inject}")
                    self.instances[classe_to_inject] = classe_to_inject(*dependency_instances_list)
                    self.dependency_injected_classes.remove(classe_to_inject)

        

    def get_dependencies_of_classes(self):
        self.dependency_mapping = {}
        for class_to_instantiate in self.dependency_injected_classes:
            s = signature(class_to_instantiate.__init__)
            self.dependency_mapping[class_to_instantiate] = []
            for parameter_name in s.parameters:    
                if(parameter_name == 'self'): continue
                parameter = s.parameters[parameter_name]
                if(parameter.default is not _empty): break
                annotation = parameter.annotation
                if(annotation is not _empty):
                    self.dependency_mapping[class_to_instantiate].append(annotation)
                else:
                    raise ValueError("You can't do dependency injection without type annotation on non defaulted values")
    
    def __pascal_to_snake(self, str_to_convert : str):
        #retirar isso daqui depois
        return re.sub("([A-Z])", 
                lambda match: '_' + match.group(1).lower(), 
                str_to_convert)[1:]

    def __enter__(self, *args):
        for class_name in self.instances:
            print(class_name.__name__)

            globals()[self.__pascal_to_snake(class_name.__name__)] = self.instances[class_name]
    
    def __exit__(self, *args):
        for class_name in self.instances:

            del globals()[self.__pascal_to_snake(class_name.__name__)]
    


In [98]:
D = DependencyInjectionContainer(MyClassDependency,  ClassToDependencyInject)

Instantiating <class '__main__.MyClassDependency'>
Instantiating <class '__main__.ClassToDependencyInject'>


In [99]:
with D: 
    print(class_to_dependency_inject.my_class_dependency is my_class_dependency)

MyClassDependency
ClassToDependencyInject
True


# Implementing intefaces

In [144]:
class IMyClass:
    pass

class Printable:
    def print(self):
        print("Hello World")

class MyClassDependency (IMyClass, Printable):
    def __init__(self ):
        pass

class IClassDependency:
    pass

class ClassToDependencyInject:
    def __init__(self, MyClassDependency : IMyClass):
        self.my_class_dependency = MyClassDependency

In [145]:
from inspect import getmro

In [146]:
getmro(ClassToDependencyInject)

(__main__.ClassToDependencyInject, object)

In [147]:
class DependencyInjectionContainer:
    '''
        This is a simple implementation of the dependency injection pattern. It's not very robust as it is not necessary for the project.
        Usually would be better to do it with a whole graph library and search for loops but I'm keeping it simple.
    '''
    DependencyInjectedClass = any
    def __init__(self, *args : DependencyInjectedClass):
        self.dependency_injected_classes : list = [*args] #convert tuple to list
        self.interface_class_mapping = {}
        self.get_dependencies_of_classes()
        self.__check_for_missing_dep()
        self.__check_for_loops()
        self.instances = {}

        self.__instantiate_classes()

    def __check_for_missing_dep(self):
        all_dependencies = []
        for class_to_inject in self.dependency_mapping:
            all_dependencies += self.dependency_mapping[class_to_inject]

        if not set(all_dependencies).issubset(self.interface_class_mapping.keys()):
            raise ValueError("Some dependencies are not inside the container")

    def __check_for_loops(self):

        #This function is extremly unoptimized but it'll do for now. 
        #If I have time I'll change it for better one.
        for interface_to_inject in self.interface_class_mapping:
            class_to_inject = self.interface_class_mapping[interface_to_inject]
            for dependency in self.dependency_mapping[class_to_inject]:
                if class_to_inject in self.dependency_mapping[self.interface_class_mapping[dependency]]:
                    raise ValueError("There is a loop in your dependencies")

    def __instantiate_classes(self):
        while len(self.dependency_injected_classes) != 0:    
            for interface_to_inject in self.interface_class_mapping:
                classe_to_inject = self.interface_class_mapping[interface_to_inject]

                if(len(self.dependency_mapping[classe_to_inject]) == 0):
                    self.instances[interface_to_inject] = classe_to_inject()
                    print(f"Instantiating {classe_to_inject}")
                    self.dependency_injected_classes.remove(classe_to_inject)

                elif(set(self.dependency_mapping[classe_to_inject]).issubset(self.instances.keys())):
                    dependency_instances_list = [self.instances[c] for c in self.dependency_mapping[classe_to_inject]]
                    print(f"Instantiating {classe_to_inject}")
                    self.instances[interface_to_inject] = classe_to_inject(*dependency_instances_list)
                    self.dependency_injected_classes.remove(classe_to_inject)

        

    def get_dependencies_of_classes(self):
        self.dependency_mapping = {}
        for class_to_instantiate in self.dependency_injected_classes:
            self.__extract_interface_mapping(class_to_instantiate)

            s = signature(class_to_instantiate.__init__)
            self.dependency_mapping[class_to_instantiate] = []
            for parameter_name in s.parameters:    
                if(parameter_name == 'self'): continue
                parameter = s.parameters[parameter_name]
                if(parameter.default is not _empty): break
                annotation = parameter.annotation
                if(annotation is not _empty):
                    self.dependency_mapping[class_to_instantiate].append(annotation)
                else:
                    raise ValueError("You can't do dependency injection without type annotation on non defaulted parameters")
                    
    def __extract_interface_mapping(self, class_to_instantiate):
        implemented_interfaces = getmro(class_to_instantiate)
        if(len(implemented_interfaces) > 1):
            implemented_interface = implemented_interfaces[1] 
            if(implemented_interface.__name__[0] == 'I'): # this is because object can appear here.
                self.interface_class_mapping[implemented_interfaces[1]] = class_to_instantiate
            else:
                self.interface_class_mapping[class_to_instantiate] = class_to_instantiate

        else:
            self.interface_class_mapping[class_to_instantiate] = class_to_instantiate

    def __pascal_to_snake(self, str_to_convert : str):
        #retirar isso daqui depois
        return re.sub("([A-Z])", 
                lambda match: '_' + match.group(1).lower(), 
                str_to_convert)[1:]

    def __enter__(self, *args):
        for class_name in self.instances:
            print(class_name.__name__)

            globals()[self.__pascal_to_snake(class_name.__name__)] = self.instances[class_name]
    
    def __exit__(self, *args):
        for class_name in self.instances:

            del globals()[self.__pascal_to_snake(class_name.__name__)]
    


In [151]:
D = DependencyInjectionContainer(MyClassDependency, ClassToDependencyInject)

with D: 
    i_my_class.print()
    #print(class_to_dependency_inject.my_class_dependency is my_class_dependency)

Instantiating <class '__main__.MyClassDependency'>
Instantiating <class '__main__.ClassToDependencyInject'>
IMyClass
ClassToDependencyInject
Hello World
